# Replikere løpet for skjema A og C fra Kompis
- Hente fra Dynarev
- I Kompis slås A og C sammen etter operasjonene er gjort på hver av datasettene. Kan det gjøres før?
- Må legge til bydeler på Oslo før de slås sammen. I kompis gjøres dette ved: identifier fylkesregion := if BYDEL <> "00" then t1.REGION || t1.BYDEL else t1.REGION,
- Oslo: kopiere alle rader med region 0301 til 0300. 
- Kan jeg kontrollere at kun gyldige arter og funksjoner er inkludert på en annen måte enn filtrering? Kanskje en merge? 

In [ ]:
import pandas as pd
import numpy as np
import glob
import requests
import io

In [ ]:
paths = glob.glob("/ssb/bruker/mtk/Testdata/inndata/*")
paths

In [ ]:
a_2021 = pd.read_csv("/ssb/bruker/mtk/Testdata/inndata/0A_2021.csv", sep = ';')
c_2021 = pd.read_csv("/ssb/bruker/mtk/Testdata/inndata/0C_2021.csv", sep = ';')

In [ ]:
#Setter sammen skjema a og c
a_c = pd.concat([a_2021, c_2021])

In [ ]:
#a_2021.dtypes

In [ ]:
#Endrer datatype. Bør kunne gjøres på alle i et steg. 
a_c['PERIODE'] = a_c['PERIODE'].astype("str")
a_c['REGION'] = a_c['REGION'].astype("str")
a_c['KONTOKLASSE'] = a_c['KONTOKLASSE'].astype("str")
a_c['FUNKSJON_KAPITTEL'] = a_c['FUNKSJON_KAPITTEL'].astype("str")
a_c['ART_SEKTOR'] = a_c['ART_SEKTOR'].astype("str")
a_c['BYDEL'] = a_c['BYDEL'].astype("str")

In [ ]:
#a_c = a_c[['PERIODE','REGION','KONTOKLASSE', 'FUNKSJON_KAPITTEL', 'ART_SEKTOR', 'BYDEL']].astype("str")
#a_c.dtypes

In [ ]:
#Velger kun de kolonnene jeg skal ha med videre
start_a_c = a_c[['PERIODE', 'REGION', 'KONTOKLASSE', 'FUNKSJON_KAPITTEL', 'ART_SEKTOR', 'BELOP', 'BYDEL']]

In [ ]:
#Endrer headingen fra store til små bokstaver
start_a_c.columns = [x.lower() for x in start_a_c.columns]

In [ ]:
#start_a_c.dtypes

In [ ]:
#Gir nytt navn til et par kolonner. Denne gir feilmelding, men funker. Bør fikses så feilmelding ikke kommer. 
start_a_c.rename(columns={'funksjon_kapittel':'funksjon','art_sektor':'art'}, inplace = True)

In [ ]:
#start_a.head()

In [ ]:
#Filtrerer bort alle rader med manglede verdier. Fungerer det? Må testes på et annet datasett som faktisk har manglende verdier... 
start_a_c.dropna()

In [ ]:
#start_a_c.head(400)

In [ ]:
#Legger på ledende nuller. Forhåpentligvis slipper vi dette når data kan hentes direkte fra ISEE
start_a_c['art']=start_a_c['art'].apply(lambda x: '{0:0>3}'.format(x))
start_a_c['region']=start_a_c['region'].apply(lambda x: '{0:0>4}'.format(x))
start_a_c['bydel']=start_a_c['bydel'].apply(lambda x: '{0:0>2}'.format(x))


### Skal endre alle verdier som har art høyere eller lik 600 til negative tall
- Koden under snur fortegn på alle verdier hvor art starter på siffer 6 eller mer. 
- Vet ikke hva advarselen betyr

In [ ]:
#start_a['belop'] = start_a.loc[np.where(start_a['art']>'600', start_a['belop'], start_a['belop']*-1)]
start_a_c['belop']=np.where((start_a_c.art > '600'), start_a_c.belop*-1, start_a_c.belop)
#start_a_c.head(400)

## Oslo bydeler og fylkeskommune
- Tallene fra Oslo skal være med både for kommune og fylkeskommune. Vi må derfor kopiere alle Oslo tall og endre region på de kopierte radene fra 0301 til 0300. 

In [ ]:
#Henter ut alle Oslo tall som har bydel. Slår sammen region og bydelkode.
bydeler = start_a_c[(start_a_c['region'] == '0301') & (start_a_c['bydel'] != '00')]
bydeler["region"] = bydeler["region"] + bydeler["bydel"]

#Fjerner bydelene fra originalfilen, slår sammen den nye bydelsfilen og arbdeisfilen. Fjerner kolonnen for bydeler. 
ac_uten_bydel = start_a_c[(start_a_c['bydel'] == '00')]
bydeler_ac = pd.concat([ac_uten_bydel, bydeler]).drop(columns=['bydel'])

In [ ]:
#Henter ut Oslo og endrer regionskode til 0300. Legger Oslo fylkeskommune på arbeidsfilen. 
oslo=bydeler_ac[(bydeler_ac['region']=='0301')]
oslo.loc[oslo['region'] == '0301', 'region'] = '0300'
forts_ac = pd.concat([bydeler_ac, oslo])
#forts_ac

## Filtrere kun gyldige funksjoner/arter
- Henter funksjons- og artslisten direkte fra KLASS via API

- Avansert: Kan vi legge kontroll_artsfilter og kontroll_funksjonsfilter under en knapp. Dvs at man tykker på knappen dersom man ønker at koden skal kjøres. 

### art

In [ ]:
#API mot KOSTRA regnskapsarter. 
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/259/codes?from=2021-01-01&to=2022-01-01&csvSeparator=;", headers=headers).content
art = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')

#Ordner datasettet til ønsket struktur
art = art[['code']]
art.rename(columns={'code':'art'}, inplace = True)
art = art[~art.art.str.contains("AG")]
#Hvorfor endrer neste linje datasettet fra frame til series?
art = art.art.replace({'A':'', 'I':'', 'D':''}, regex=True).drop_duplicates()
art = art.to_frame()
art


In [ ]:
#Merger alle gyldige arter med regnskapsfilen. Dette vil fjerne alle rader med ugyldige arter. 
artsfilter=pd.merge(art, forts_ac, on=['art'])

In [ ]:
#Kontrollerer om noen arter ble fjernet. Er test tom er ingen rader filtrer ut i forrige steg og det var dermed ikke inkludert noen ugyldige arter. 
kontroll_artsfilter = forts_ac.merge(artsfilter,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
kontroll_artsfilter

### funksjon

In [ ]:
#API mot KOSTRA regnskapsfunksjoner
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/277/codes?from=2021-01-01&to=2022-01-01&csvSeparator=;", headers=headers).content
funksjon = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')

#Ordner datasettet til ønsket struktur
funksjon = funksjon[['code']]
funksjon.rename(columns={'code':'funksjon'}, inplace = True)
funksjon = funksjon[~funksjon.funksjon.str.contains("F")]
#funksjon

In [ ]:
funksjonsfilter = pd.merge(funksjon, artsfilter, on=['funksjon'])

In [ ]:
#Kontrollerer om noen funksjoner ble fjernet. Er test tom er ingen rader filtrer ut i forrige steg og det var dermed ikke inkludert noen ugyldige arter. 
kontroll_funksjonsfilter = artsfilter.merge(funksjonsfilter,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
kontroll_funksjonsfilter

## Finner hvilke kommuner som har levert og hvem som mangler skjema 0A og 0C
- For å finne hvem som mangler må vi koble opp mot regionslisten. Dette vil jeg også gjøre via et API mot Klass. 

- Lagt opp mot kommuner og fylkeskommuner, ikke bydeler


In [ ]:
levert = funksjonsfilter[['region']].drop_duplicates()
#levert

### Kommuner

In [ ]:
#API mot KOSTRA-kommuner med tilhørende regionsgrupperinger
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/231/codes?from=2021-01-01&to=2022-01-01&csvSeparator=;", headers=headers).content
kommuner = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')
kommuner = kommuner[['code']]
kommuner.rename(columns={'code':'region'}, inplace = True)
kommuner = kommuner[~kommuner.region.str.contains("E")]
#kommuner

### Fylkeskommuner

In [ ]:
#API mot KOSTRA-fylkeskommuner med tilhørende regionsgrupperinger
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/232/codes?from=2021-01-01&to=2022-01-01&csvSeparator=;", headers=headers).content
fykomm = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')
fykomm = fykomm[['code']]
fykomm.rename(columns={'code':'region'}, inplace = True)
fykomm = fykomm[~fykomm.region.str.contains("E")]
#fykomm

### Slår sammen til alle regioner

In [ ]:
regioner = pd.concat([kommuner, fykomm])

### Lister ut kun de som ikke er inkludert i "levert"

In [ ]:
manglende = regioner.merge(levert,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
manglende

# Henter inn fjorårets regnskap for å imputere manglende data

In [ ]:
#Henter inn parquet-fil
a_c_2020 = pd.read_parquet("../../../../Testdata/arbeidsdata/a_c_2020.parquet")

In [ ]:
a_c_2020.rename(columns={'fylkesregion':'region'}, inplace = True)
estimat_artsfilter=pd.merge(art, a_c_2020, on=['art'])
estimat_funksjonsfilter=pd.merge(funksjon, estimat_artsfilter, on=['funksjon'])
a_c_2020 = estimat_funksjonsfilter
a_c_2020

In [ ]:
manglende_estimat=pd.merge(manglende, a_c_2020, on=['region']).drop(columns=['_merge'])
manglende_estimat.loc[manglende_estimat['periode'] == '2020', 'periode'] = '2021'
manglende_estimat

In [ ]:
#Setter sammen estimert og levert
a_c_basis = pd.concat([funksjonsfilter, manglende_estimat])

In [ ]:
a_c_basis

In [ ]:
a_c_basis['regnskapsomfang'] = 'B'
#a_c_basis

In [ ]:
a_c_basis.to_parquet("../../../../Testdata/arbeidsdata/a_c_basis_2021.parquet")